# CCSD calculations

In [1]:
# Force the local gqcpy to be imported
import sys
sys.path.insert(0, '../../build/gqcpy/')

import gqcpy
import numpy as np

np.set_printoptions(precision=3, linewidth=120)

## CCSD calculations on H2O // STO-3G

In this example, we will try to reproduce the [crawdad CCSD results](https://github.com/CrawfordGroup/ProgrammingProjects/tree/master/Project%2305).

In [2]:
molecule = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2o_crawdad.xyz")
N = molecule.numberOfElectrons()
N_P = molecule.numberOfElectronPairs()

print(molecule)

Number of electrons: 10 
O  (0, -0.143226, 0)
H  (1.63804, 1.13655, -0)
H  (-1.63804, 1.13655, -0)



The first task is to generate the RHF reference, so we'll do an RHF SCF calculation.

In [3]:
spin_orbital_basis = gqcpy.RSpinOrbitalBasis_d(molecule, "STO-3G")
K = spin_orbital_basis.numberOfSpatialOrbitals()

S = spin_orbital_basis.quantize(gqcpy.OverlapOperator())

In [4]:
hamiltonian = gqcpy.RSQHamiltonian_d.Molecular(spin_orbital_basis, molecule)

The current Hamiltionian is expressed in the non-orthogonal AO basis. This is exactly what we need to start an RHF calculation.

In [5]:
environment = gqcpy.RHFSCFEnvironment_d.WithCoreGuess(N, hamiltonian, S)
solver = gqcpy.RHFSCFSolver_d.DIIS()

objective = gqcpy.DiagonalRHFFockMatrixObjective_d(hamiltonian)  # use the default threshold of 1.0e-08

Using this objective makes sure that the optimized expansion coefficients yield a diagonal Fock matrix, so they will represent the canonical RHF spin-orbital basis.

In [6]:
rhf_qc_structure = gqcpy.RHF_d.optimize(objective, solver, environment)

In [7]:
rhf_energy = rhf_qc_structure.groundStateEnergy() + gqcpy.NuclearRepulsionOperator(molecule.nuclearFramework()).value()
print(rhf_energy)

-74.94207992809227


This is very much in line with `crawdad`'s results: `-74.942079928192`.

In [8]:
rhf_parameters = rhf_qc_structure.groundStateParameters()
C = rhf_parameters.expansion()

Now, let's transform the spin-orbital basis and the second-quantized Hamiltonian to the RHF MOs.

In [9]:
spin_orbital_basis.transform(C)  # Now represents a restricted spin-orbital that contains the canonical RHF MOs.
print(spin_orbital_basis.expansion().matrix())

[[-9.944e-01 -2.392e-01  2.541e-16 -9.368e-02  8.797e-31  1.116e-01 -2.116e-16]
 [-2.410e-02  8.857e-01 -1.130e-15  4.796e-01 -4.496e-30 -6.696e-01  1.316e-15]
 [ 4.352e-21 -4.532e-16 -6.073e-01 -1.198e-15 -6.823e-18 -2.417e-15 -9.192e-01]
 [-3.162e-03  8.590e-02  2.529e-15 -7.474e-01  7.030e-30 -7.385e-01  1.054e-15]
 [-6.491e-34  8.228e-32 -1.191e-16 -9.744e-30 -1.000e+00  4.216e-31  1.084e-16]
 [ 4.594e-03  1.440e-01 -4.530e-01 -3.295e-01  6.937e-17  7.098e-01  7.325e-01]
 [ 4.594e-03  1.440e-01  4.530e-01 -3.295e-01 -6.937e-17  7.098e-01 -7.325e-01]]


In [10]:
hamiltonian.transform(C)
print(hamiltonian.core().parameters())

[[-3.256e+01 -5.683e-01  6.009e-16 -2.095e-01  1.990e-30  2.563e-01 -5.109e-16]
 [-5.683e-01 -7.564e+00 -3.062e-16 -5.223e-01  5.652e-30  1.220e+00 -3.060e-15]
 [ 6.112e-16 -7.801e-17 -6.018e+00  3.826e-15 -5.569e-16 -1.964e-15 -1.767e+00]
 [-2.095e-01 -5.223e-01  4.025e-15 -6.610e+00 -8.581e-30 -1.306e+00 -2.575e-15]
 [ 1.999e-30  5.606e-30 -5.569e-16 -8.587e-30 -7.347e+00  1.543e-29  6.278e-16]
 [ 2.563e-01  1.220e+00 -2.399e-15 -1.306e+00  1.537e-29 -5.291e+00 -4.069e-15]
 [-5.326e-16 -3.269e-15 -1.767e+00 -2.500e-15  6.278e-16 -3.469e-15 -5.513e+00]]


Before continuing with the CCSD specifics, we must first prepare the molecular Hamiltonian in the correct spinor basis. Since we have implemented CCSD using general spinors, we should use a `GSpinorBasis`.

In [11]:
spinor_basis = gqcpy.GSpinorBasis_d.FromRestricted(spin_orbital_basis)  # Represents a general spinor basis, based off the restricted canonical RHF MOs.

# We can inspect the two non-zero blocks (top-left and bottom-right) of the coefficient matrix.
print(spinor_basis.expansion().matrix())

[[-9.944e-01 -2.392e-01  2.541e-16 -9.368e-02  8.797e-31  1.116e-01 -2.116e-16  0.000e+00  0.000e+00  0.000e+00
   0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [-2.410e-02  8.857e-01 -1.130e-15  4.796e-01 -4.496e-30 -6.696e-01  1.316e-15  0.000e+00  0.000e+00  0.000e+00
   0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 4.352e-21 -4.532e-16 -6.073e-01 -1.198e-15 -6.823e-18 -2.417e-15 -9.192e-01  0.000e+00  0.000e+00  0.000e+00
   0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [-3.162e-03  8.590e-02  2.529e-15 -7.474e-01  7.030e-30 -7.385e-01  1.054e-15  0.000e+00  0.000e+00  0.000e+00
   0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [-6.491e-34  8.228e-32 -1.191e-16 -9.744e-30 -1.000e+00  4.216e-31  1.084e-16  0.000e+00  0.000e+00  0.000e+00
   0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 4.594e-03  1.440e-01 -4.530e-01 -3.295e-01  6.937e-17  7.098e-01  7.325e-01  0.000e+00  0.000e+00  0.000e+00
   0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 4.594e-03  1.440e-01  4.530e-01 -3.295e-01 

We should note that the ordering of the MOs in this spinor basis is not by ascending energy, but they are ordered alphas first and then all betas.

For H2O, we have 10 electrons, so this means we shouldn't occupy the first 10 spinors (since that would correspond to occupying all alpha-spin-orbitals and then 3 beta-spin-orbitals), but we should occupy the first 5 alpha spin-orbitals and the first 5 beta-spin-orbitals.

This issue is circumvented by constructing an `OrbitalSpace`, which takes care of setting up the occupied and virtual orbital spaces. We don't have to construct this `OrbitalSpace` by hand: GQCP provides an API through the creation of the 'GHF' reference ONV.

In [12]:
M = spinor_basis.numberOfSpinors()
ghf_onv = gqcpy.SpinUnresolvedONV.GHF(M, N, rhf_parameters.spinOrbitalEnergiesBlocked())

orbital_space = ghf_onv.orbitalSpace()

In [13]:
print(orbital_space.description())

An orbital space with 14 orbitals.

	The occupied orbital indices are: 0 1 2 3 4 7 8 9 10 11 
	The active orbital indices are: 
	The virtual orbital indices are: 5 6 12 13 


CCSD is also implemented as a `QCMethod`. This means that we'll have to set up an environment and a particular solver.

In [14]:
hamiltonian = gqcpy.GSQHamiltonian_d.Molecular(spinor_basis, molecule)

In [15]:
solver = gqcpy.CCSDSolver.Plain()
environment = gqcpy.CCSDEnvironment.PerturbativeCCSD(hamiltonian, orbital_space)

It's interesting to see that the initial, perturbative, T1 amplitudes are zero, which is a direct consequence of Brillouin's theorem: the occupied-virtual block of the Fock(ian) matrix is zero!

In [16]:
print(environment.t1_amplitudes[-1].asMatrix())

[[ 9.239e-15 -1.921e-18 -0.000e+00 -0.000e+00]
 [ 1.754e-13  1.266e-16 -0.000e+00 -0.000e+00]
 [ 8.802e-16 -7.514e-13 -0.000e+00 -0.000e+00]
 [-7.726e-13 -2.890e-17 -0.000e+00 -0.000e+00]
 [-1.896e-31 -3.917e-17 -0.000e+00 -0.000e+00]
 [-0.000e+00 -0.000e+00  9.238e-15 -1.921e-18]
 [-0.000e+00 -0.000e+00  1.754e-13  1.266e-16]
 [-0.000e+00 -0.000e+00  8.802e-16 -7.515e-13]
 [-0.000e+00 -0.000e+00 -7.728e-13 -2.890e-17]
 [-0.000e+00 -0.000e+00 -1.896e-31 -3.917e-17]]


The initial T2-amplitudes are actually the MP2 T2-amplitudes, so the initial CCSD correlation energy is the MP2 correlation energy. Crawdad reports a value of `-0.049149636120`.

In [17]:
print(environment.correlation_energies[-1])

-0.04914963614941137


Let's now proceed by actually optimizing the CCSD wave function model. This may take a while if `gqcp` is built in Debug mode!

In [18]:
ccsd_qc_structure = gqcpy.CCSD.optimize(solver, environment)

Crawdad lists a converged CCSD energy as: `-0.070680088376`.

In [19]:
ccsd_correlation_energy = ccsd_qc_structure.groundStateEnergy()
print(ccsd_correlation_energy)

-0.07068008793709481
